In [141]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np


In [142]:
df = pd.read_csv("../data/CleanedFlights.csv")
print(df[df["ORIGIN_AIRPORT"] == "ANC"])
# df.head()

C:\Users\Florian\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3369: DtypeWarning:

Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.



         Unnamed: 0   FULL_DATE  YEAR  MONTH  DAY DAY_OF_WEEK AIRLINE  \
0                 0  2015-01-01  2015      1    1    Thursday      AS   
15               15  2015-01-01  2015      1    1    Thursday      AS   
16               16  2015-01-01  2015      1    1    Thursday      DL   
18               18  2015-01-01  2015      1    1    Thursday      AS   
32               32  2015-01-01  2015      1    1    Thursday      AS   
...             ...         ...   ...    ...  ...         ...     ...   
5818969     5818969  2015-12-31  2015     12   31    Thursday      AS   
5818977     5818977  2015-12-31  2015     12   31    Thursday      AS   
5819012     5819012  2015-12-31  2015     12   31    Thursday      DL   
5819015     5819015  2015-12-31  2015     12   31    Thursday      UA   
5819037     5819037  2015-12-31  2015     12   31    Thursday      AS   

        ORIGIN_AIRPORT DESTINATION_AIRPORT SCHEDULED_DEPARTURE DEPARTURE_TIME  \
0                  ANC                 SEA

In [143]:
df["ARRIVAL_DELAY"] = df["ARRIVAL_DELAY"].astype(float)
count = pd.DataFrame(df.groupby(["ORIGIN_AIRPORT"]).count()["YEAR"])
df = df.groupby(["ORIGIN_AIRPORT"]).mean()
preparedDf = pd.DataFrame({
    "TOTAL_FLIGHTS" : count["YEAR"], 
    "AVERAGE_ARRIVAL_DELAY": df.ARRIVAL_DELAY })
preparedDf = preparedDf.reset_index(drop= False)
preparedDf.tail(50)

,ORIGIN_AIRPORT,TOTAL_FLIGHTS,AVERAGE_ARRIVAL_DELAY
880,SBN,4454,8.896845
881,SBP,3133,4.552073
882,SCC,855,1.099265
883,SCE,911,6.103175
884,SDF,10204,3.248189
885,SEA,110899,2.975567
886,SFO,148008,5.750211
887,SGF,6236,9.515659
888,SGU,1777,1.873446
889,SHV,6099,9.921443


In [151]:
fig = go.Figure()
scatter = go.Scatter(x= preparedDf["TOTAL_FLIGHTS"], y = preparedDf["AVERAGE_ARRIVAL_DELAY"], mode = "markers", name = "",
                    customdata = preparedDf["ORIGIN_AIRPORT"],
                    hovertemplate = "%{customdata}")
fig.add_trace(scatter)
fig.show()
